In [2]:
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env
from tasho import WorldSimulator
from tasho.OCPGenerator import OCPGenerator
from robotshyu import Robot as rob
from time import sleep
import numpy as np
import casadi as cs
import pybullet as p
import tasho
import robotshyu
import os

pybullet build time: May 20 2022 19:44:17


In [3]:
# Options for example codes
frame_enable = True

# Ex1) P2P example

In [3]:
robot = rob.Robot("indy7")
link_name = 6
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[0, 0, 1, 1], [0, 1, 0, -0.1], [-1, 0, 0, 0.6], [0, 0, 0, 1]]
    ))
q0 = [1.0, -0.05311582280659044, -0.815452580946695, -1.3191046402052224, -0.8582660722530533, 1.3988994390898029]

robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.nq
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 6-DoF robot: indy7


In [4]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [5]:
task_P2P.write_task_graph("Indy7_P2P_Task.svg")

In [6]:
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1),
                              Regularization(task_P2P.variables['qdd_'+robot.name], 0.1))

In [7]:
task_P2P.write_task_graph("Indy7_P2P_Task2.svg")

In [8]:
horizon_steps = 10
horizon_period = 3.0
q_pred = [0]*horizon_steps

OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      452
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:      141

Total number of variables............................:      192
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      141
Total number of inequality constrai

In [9]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

obj = WorldSimulator.WorldSimulator()

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]

robotID = obj.add_robot(position,orientation,robot_pkg=robot)
frameID0 = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)
robotIDs=[robotID,frameID0]

if frame_enable:
    frameIDs = [0]*horizon_steps
    for i in range(horizon_steps):
        frameIDs[i] = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)


for i in range(horizon_steps):
    q_pred[i]=qsol[i]

joint_indices = [0, 1, 2, 3, 4, 5]

obj.resetMultiJointState(robotIDs, joint_indices, [q0])

if frame_enable:
    obj.resetMultiJointState(frameIDs, joint_indices, [q0])
    obj.resetMultiJointState(frameIDs,joint_indices,q_pred)

In [10]:
for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetMultiJointState(
        robotIDs, joint_indices, [qsol[i]]
    )
    
sleep(0.5)
obj.end_simulation()

Ending simulation


# Ex2) Object picking MPC

In [4]:
robot = rob.Robot("indy7")
link_name = 6

# Define initial conditions of the robot
q0_val = [0, -0.423598, -0.9, 0.0, -1.5709, -0.523598]
qd0_val = [0] * robot.ndof

robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, link_name, goal_pose, q_init, 0.001)
task_P2P.write_task_graph("Indy7_P2P_Task4.svg")

Loading robot params from json ...
Loaded 6-DoF robot: indy7


In [5]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_indy7'])
task_P2P.remove_terminal_constraints('rot_con_pose_6_indy7_vs_goal',
                                    'trans_con_pose_6_indy7_vs_goal')
task_P2P.write_task_graph("Indy7_P2P_Task5.svg")

In [6]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_indy7']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_6_indy7_vs_goal'], 5e0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_6_indy7_vs_goal'], 1e0), 
                            Regularization(task_P2P.variables['qd_indy7'], 1e-1),
                            Regularization(task_P2P.variables['qdd_indy7'], 1e-3))

task_P2P.write_task_graph("Indy7_P2P_Task6.svg")

In [7]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 5,"tol": 1e-3}})
# tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 0,"tol": 1e-3, "linear_solver":"ma27"}}) #use this if you have hsl


################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_indy7'].x
qd = pOCP.stage_tasks[0].variables['qd_indy7'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_indy7'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_indy7'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_indy7'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([-1, 0, 0, 0.5]),
    cs.hcat([0, 1, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_indy7", "var":"qd_indy7", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_6_indy7_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

mpc_options = default_mpc_options.get_default_mpc_options()


tc.ocp_solver = "ipopt"
tc.ocp_options = mpc_options["ipopt"]["options"]
tc.mpc_solver = tc.ocp_solver
tc.mpc_options = tc.ocp_options
tc.set_ocp_solver("ipopt", tc.mpc_options)
os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  70.00us ( 23.33us)  70.48us ( 23.49us)         3
       nlp_g  |  27.00us (  9.00us)  27.38us (  9.13us)         3
    nlp_grad  |  86.00us ( 86.00us)  85.81us ( 85.81us)         1
  nlp_grad_f  | 189.00us ( 63.00us) 190.15us ( 63.38us)         3
  nlp_hess_l  | 646.00us (323.00us) 645.96us (322.98us)         2
   nlp_jac_g  |  33.00us ( 11.00us)  32.18us ( 10.73us)         3
       total  |   3.47ms (  3.47ms)   3.47ms (  3.47ms)         1


sh: 1: export: : bad variable name


In [8]:
dir_casadi_func = "casadi_dir"
os.makedirs("./"+dir_casadi_func, exist_ok=True)
vars_db = tc.generate_MPC_component("./"+dir_casadi_func+"/", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti0_p_2[12:15]-fk_T(opti0_x_1){6}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti0_p_2[12:15]-fk_T(opti0_x_1){6}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [9]:
MPC_component = MPC("indy7_obj_pickup", "./"+dir_casadi_func+"/"+ tc.name + ".json")

In [10]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]

robotID = obj.add_robot(position,orientation,robot_pkg=robot)

if frame_enable:
    frameIDs = [0]*horizon_size
    for i in range(horizon_size):
        frameIDs[i] = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 3, 4, 5]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")

obj.resetJointState(robotID,joint_indices,q0_val)
if frame_enable==True:
    obj.resetMultiJointState(frameIDs, joint_indices, [q0_val])

obj.setController(
    robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)
q_log = []
q_dot_log = []
predicted_pos_log = []
q_pred = [0]*horizon_size


In [11]:
# Execute the MPC loop

for i in range(horizon_size * 30):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(robotID, joint_indices)
    qd_now = obj.readJointVelocities(robotID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_indy7"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_indy7"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([-1, 0, 0, predicted_pos[0]]),
        cs.hcat([0, 1, 0, predicted_pos[1]]),
        cs.hcat([0, 0, -1, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_indy7"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()
    if frame_enable==True:
        sol = MPC_component.res_vals
        for i in range(horizon_size):
            q_pred[i]=sol[robot.ndof*i:robot.ndof*i+robot.ndof].full()
        obj.resetMultiJointState(frameIDs, joint_indices, q_pred)

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_indy7"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_indy7"]["val"] * t_mpc).full()
    
    obj.setController(
        robotID, "velocity", joint_indices, targetVelocities=qd_control_sig+qdd_control_sig
    )
    
    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
#     if "termination_criteria_true" in MPC_component.event_output_port:
#         break

----------- MPC execution -----------
Predicted position of cube [0.5, 0.15777500346789, 0.41]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 123.00us ( 30.75us) 124.51us ( 31.13us)         4
       nlp_g  |  33.00us (  8.25us)  33.75us (  8.44us)         4
    nlp_grad  | 104.00us (104.00us) 103.67us (103.67us)         1
  nlp_grad_f  | 305.00us ( 76.25us) 305.95us ( 76.49us)         4
  nlp_hess_l  |   1.14ms (380.67us)   1.14ms (380.25us)         3
   nlp_jac_g  |  56.00us ( 14.00us)  54.83us ( 13.71us)         4
       total  |  23.32ms ( 23.32ms)   7.28ms (  7.28ms)         1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  28.00us ( 28.00us)  28.46us ( 28.46us)         1
       nlp_g  |  10.00us ( 10.00us)  10.55us ( 10.55us)         1
    nlp_grad  | 101.00us (101.00us) 101.88us (101.88us)         1
  nlp_grad_f  |  77.00us ( 77.00us)  78.31us ( 78.31us)         1
   nlp_jac_g  |  13.00us ( 13.00us)  13.29us ( 

----------- MPC execution -----------
Predicted position of cube [0.4999996894332, 0.30652947968197, 0.27249342079112]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 100.00us ( 33.33us)  99.91us ( 33.30us)         3
       nlp_g  |  35.00us ( 11.67us)  34.24us ( 11.41us)         3
    nlp_grad  | 124.00us (124.00us) 124.75us (124.75us)         1
  nlp_grad_f  | 254.00us ( 84.67us) 254.68us ( 84.89us)         3
  nlp_hess_l  |   1.35ms (676.00us) 818.27us (409.14us)         2
   nlp_jac_g  |  46.00us ( 15.33us)  45.73us ( 15.24us)         3
       total  |  29.73ms ( 29.73ms)   4.08ms (  4.08ms)         1
----------- MPC execution -----------
Predicted position of cube [0.499999682618, 0.31677327478954, 0.2724911803854]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 103.00us ( 34.33us) 102.85us ( 34.28us)         3
       nlp_g  |  38.00us ( 12.67us)  37.57us ( 12.52us)         3
    nlp_grad  | 122.00us (122.00us) 12

----------- MPC execution -----------
Predicted position of cube [0.49999814352022, 0.42804380177726, 0.084991218390446]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 100.00us ( 33.33us) 100.98us ( 33.66us)         3
       nlp_g  |  42.00us ( 14.00us)  41.42us ( 13.81us)         3
    nlp_grad  | 266.00us (266.00us) 117.02us (117.02us)         1
  nlp_grad_f  |   3.95ms (  1.32ms) 247.81us ( 82.60us)         3
  nlp_hess_l  |   1.98ms (990.00us) 809.93us (404.97us)         2
   nlp_jac_g  |  42.00us ( 14.00us)  42.87us ( 14.29us)         3
       total  |  16.61ms ( 16.61ms)   3.93ms (  3.93ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999813876991, 0.43079058463232, 0.084990125141273]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 110.00us ( 36.67us) 110.50us ( 36.83us)         3
       nlp_g  |  35.00us ( 11.67us)  34.49us ( 11.50us)         3
    nlp_grad  | 102.00us (102.00

----------- MPC execution -----------
Predicted position of cube [0.49999413696363, 0.43632680946756, 0.084989996518309]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  95.00us ( 31.67us)  94.73us ( 31.58us)         3
       nlp_g  |  36.00us ( 12.00us)  36.27us ( 12.09us)         3
    nlp_grad  | 112.00us (112.00us) 108.70us (108.70us)         1
  nlp_grad_f  | 789.00us (263.00us) 239.18us ( 79.73us)         3
  nlp_hess_l  |   3.72ms (  1.86ms) 823.43us (411.72us)         2
   nlp_jac_g  |  42.00us ( 14.00us)  41.75us ( 13.92us)         3
       total  |  12.58ms ( 12.58ms)   4.05ms (  4.05ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413871289, 0.43632680975236, 0.084989998719897]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 104.00us ( 34.67us) 104.45us ( 34.82us)         3
       nlp_g  |   1.07ms (358.33us)  42.14us ( 14.05us)         3
    nlp_grad  | 109.00us (109.00

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 103.00us ( 34.33us) 103.01us ( 34.34us)         3
       nlp_g  |  32.00us ( 10.67us)  31.72us ( 10.57us)         3
    nlp_grad  | 102.00us (102.00us) 102.32us (102.32us)         1
  nlp_grad_f  | 259.00us ( 86.33us) 259.64us ( 86.55us)         3
  nlp_hess_l  | 874.00us (437.00us) 874.85us (437.42us)         2
   nlp_jac_g  |  45.00us ( 15.00us)  43.52us ( 14.51us)         3
       total  |  12.37ms ( 12.37ms)   4.29ms (  4.29ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 113.00us ( 37.67us) 110.54us ( 36.85us)         3
       nlp_g  |   3.58ms (  1.19ms)  40.32us ( 13.44us)         3
    nlp_grad  |   4.94ms (  4.94

...

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  94.00us ( 31.33us)  94.13us ( 31.38us)         3
       nlp_g  |  30.00us ( 10.00us)  29.92us (  9.97us)         3
    nlp_grad  | 123.00us (123.00us) 122.52us (122.52us)         1
  nlp_grad_f  | 241.00us ( 80.33us) 241.50us ( 80.50us)         3
  nlp_hess_l  |   8.83ms (  4.41ms) 822.30us (411.15us)         2
   nlp_jac_g  |  44.00us ( 14.67us)  43.77us ( 14.59us)         3
       total  |  12.17ms ( 12.17ms)   4.16ms (  4.16ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 100.00us ( 33.33us) 100.16us ( 33.39us)         3
       nlp_g  |  38.00us ( 12.67us)  37.84us ( 12.61us)         3
    nlp_grad  |   6.22ms (  6.22

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 195.00us ( 65.00us)  80.60us ( 26.87us)         3
       nlp_g  |  23.00us (  7.67us)  22.72us (  7.57us)         3
    nlp_grad  |  98.00us ( 98.00us)  97.84us ( 97.84us)         1
  nlp_grad_f  | 200.00us ( 66.67us) 200.70us ( 66.90us)         3
  nlp_hess_l  |   2.58ms (  1.29ms) 669.96us (334.98us)         2
   nlp_jac_g  |  35.00us ( 11.67us)  35.39us ( 11.80us)         3
       total  |  14.88ms ( 14.88ms)   3.55ms (  3.55ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  92.00us ( 30.67us)  93.11us ( 31.04us)         3
       nlp_g  |  36.00us ( 12.00us)  36.79us ( 12.26us)         3
    nlp_grad  |  91.00us ( 91.00

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.16ms (386.67us)  94.57us ( 31.52us)         3
       nlp_g  |  30.00us ( 10.00us)  29.04us (  9.68us)         3
    nlp_grad  | 183.00us (183.00us) 182.52us (182.52us)         1
  nlp_grad_f  | 242.00us ( 80.67us) 243.43us ( 81.14us)         3
  nlp_hess_l  | 819.00us (409.50us) 820.82us (410.41us)         2
   nlp_jac_g  |  51.00us ( 17.00us)  50.59us ( 16.86us)         3
       total  |  14.64ms ( 14.64ms)   3.94ms (  3.94ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  97.00us ( 32.33us)  96.46us ( 32.15us)         3
       nlp_g  |  33.00us ( 11.00us)  33.26us ( 11.09us)         3
    nlp_grad  | 111.00us (111.00

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  91.00us ( 30.33us)  92.21us ( 30.74us)         3
       nlp_g  |  35.00us ( 11.67us)  32.54us ( 10.85us)         3
    nlp_grad  | 148.00us (148.00us) 148.49us (148.49us)         1
  nlp_grad_f  | 230.00us ( 76.67us) 231.89us ( 77.30us)         3
  nlp_hess_l  |  30.73ms ( 15.37ms) 795.29us (397.65us)         2
   nlp_jac_g  |  44.00us ( 14.67us)  43.33us ( 14.44us)         3
       total  |  36.71ms ( 36.71ms)   4.42ms (  4.42ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  99.00us ( 33.00us)  98.84us ( 32.95us)         3
       nlp_g  |  39.00us ( 13.00us)  38.63us ( 12.88us)         3
    nlp_grad  | 112.00us (112.00

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 296.00us ( 98.67us)  82.41us ( 27.47us)         3
       nlp_g  |  33.00us ( 11.00us)  33.12us ( 11.04us)         3
    nlp_grad  |   2.51ms (  2.51ms)  99.40us ( 99.40us)         1
  nlp_grad_f  | 328.00us (109.33us) 221.08us ( 73.69us)         3
  nlp_hess_l  | 708.00us (354.00us) 710.92us (355.46us)         2
   nlp_jac_g  |  41.00us ( 13.67us)  40.83us ( 13.61us)         3
       total  |  25.62ms ( 25.62ms)   4.18ms (  4.18ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  94.00us ( 31.33us)  94.16us ( 31.39us)         3
       nlp_g  |  32.00us ( 10.67us)  29.70us (  9.90us)         3
    nlp_grad  | 115.00us (115.00

In [12]:
obj.end_simulation()

max(MPC_component.solver_times)

Ending simulation


0.01711273193359375

In [13]:
qd_control_sig+qdd_control_sig

array([[ 7.66154574e-05],
       [ 1.48693177e-04],
       [-4.72362471e-04],
       [-1.56550123e-04],
       [ 4.36071320e-04],
       [ 1.92589937e-04]])